In [1]:
import pandas as d
import numpy as np
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
from ModelMethods import *
from tqdm import tqdm

In [2]:
with open('data/movement_df_with_ld_validation.pkl', 'rb') as f:
    movement_df = pickle.load(f)
movement_df

,author_id,move_year,institution_id,move_time,prev_ins,stay_time,city,country_code,continent_code,prev_country,level_distance,work,coauthors,field,domain,author_validated
0,A5000000072,2019,I198243066,2019-01-01,I153718931,NaN,Heredia,CR,NaN,CA,2,0,0,2,1,3
1,A5000000092,2002,I74760111,2002-11-23,I131249849,4057 days 00:00:00,Dallas,US,NaN,US,1,0,0,1,1,2
2,A5000000092,2014,I16076960,2014-01-01,I74760111,2475 days 00:00:00,Islamabad,PK,AS,US,3,0,0,1,1,2
3,A5000000092,2020,I131249849,2020-10-11,I16076960,NaN,Corvallis,US,NaN,PK,3,1,5,1,1,8
4,A5000000103,1992,I4210135003,1992-05-01,I88155538,11644 days 00:00:00,Pamplona,ES,EU,ES,0,0,1,2,2,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5194551,A5097964981,2004,I4210139845,2004-03-01,I904495901,NaN,Bochum,DE,EU,DE,0,0,0,2,3,5
5194552,A5097991687,1993,I2801112126,1993-08-01,I4210122458,NaN,Bern,CH,EU,CH,0,0,0,2,2,4
5194553,A5098096471,1985,I4210159963,1985-11-01,I2801942218,NaN,Copenhagen,DK,EU,DK,1,1,0,1,2,4
5194554,A5098805673,2015,I80043,2015-12-01,I4210147769,NaN,Paris,FR,EU,ES,2,23,0,3,3,29


In [3]:
movement_df['verifiation_score'] = movement_df['work'] + movement_df['coauthors'] + movement_df['field']
verified_df = movement_df[movement_df['verifiation_score'] > 0 ]
print(len(verified_df) / len(movement_df))
move_data = verified_df
move_data

0.9927653104519424


,author_id,move_year,institution_id,move_time,prev_ins,stay_time,city,country_code,continent_code,prev_country,level_distance,work,coauthors,field,domain,author_validated,verifiation_score
0,A5000000072,2019,I198243066,2019-01-01,I153718931,NaN,Heredia,CR,NaN,CA,2,0,0,2,1,3,2
1,A5000000092,2002,I74760111,2002-11-23,I131249849,4057 days 00:00:00,Dallas,US,NaN,US,1,0,0,1,1,2,1
2,A5000000092,2014,I16076960,2014-01-01,I74760111,2475 days 00:00:00,Islamabad,PK,AS,US,3,0,0,1,1,2,1
3,A5000000092,2020,I131249849,2020-10-11,I16076960,NaN,Corvallis,US,NaN,PK,3,1,5,1,1,8,7
4,A5000000103,1992,I4210135003,1992-05-01,I88155538,11644 days 00:00:00,Pamplona,ES,EU,ES,0,0,1,2,2,5,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5194551,A5097964981,2004,I4210139845,2004-03-01,I904495901,NaN,Bochum,DE,EU,DE,0,0,0,2,3,5,2
5194552,A5097991687,1993,I2801112126,1993-08-01,I4210122458,NaN,Bern,CH,EU,CH,0,0,0,2,2,4,2
5194553,A5098096471,1985,I4210159963,1985-11-01,I2801942218,NaN,Copenhagen,DK,EU,DK,1,1,0,1,2,4,2
5194554,A5098805673,2015,I80043,2015-12-01,I4210147769,NaN,Paris,FR,EU,ES,2,23,0,3,3,29,26


In [5]:
with open('data/institution_geo24.pkl', 'rb') as f:
    institution_geo = pickle.load(f)

institution_geo.dropna(subset=["city", 'country_code'], inplace=True)

ccframe = pd.read_csv('data/country-and-continent-codes-list-csv.csv')

def get_continent(country_code):
    
    if country_code == 'XK':
        con = 'EU'
    else:
        con =  str(ccframe.loc[ccframe['Two_Letter_Country_Code'] == country_code ]['Continent_Code'].values[0])
    if(con != 'nan'):
        return con
    else:
        return'NA'
        ## Pandas 好像会自动把美洲的code ”NA“ 转化成NaN，所以要特判一下
        
institution_geo['continent_code'] = institution_geo['country_code'].apply(get_continent)

# Change all TW to CN in the country_code column 
institution_geo.loc[institution_geo['country_code'] == 'TW', 'country_code'] = 'CN'
# Change all Taiwan to China in the country column
institution_geo.loc[institution_geo['country'] == 'Taiwan', 'country'] = 'China'

In [6]:
destinations = move_data['institution_id'].unique()
origins = move_data['prev_ins'].unique()

institution_geo = institution_geo[institution_geo['institution_id'].isin(destinations)|
                                  institution_geo['institution_id'].isin(origins)]
institution_geo

,institution_id,city,region,country_code,country,latitude,longitude,continent_code
484,I4210142806,Boone,North Carolina,US,United States,36.199036,-81.652270,NA
485,I4210103515,Rubano,NaN,IT,Italy,45.425476,11.789450,EU
487,I4210130444,Manchester,Missouri,US,United States,38.596786,-90.509660,NA
489,I4210117313,San Diego,California,US,United States,32.714620,-117.151380,NA
490,I4210124511,Philadelphia,Pennsylvania,US,United States,39.952330,-75.163790,NA
...,...,...,...,...,...,...,...,...
107143,I4210138289,Milton,Queensland,AU,Australia,-27.467762,153.006040,OC
107191,I4210143133,New Delhi,NaN,IN,India,28.624348,77.191010,AS
107348,I4210159722,Arlington,NaN,US,United States,38.881010,-77.104280,NA
107380,I4210163951,Jilin City,NaN,CN,China,43.850830,126.560280,AS
